In [142]:
import train as train
import time
import scipy.sparse
import pycrfsuite as crf
import helper

training_file = "../data/tagged_data/whole_text_full_city2/train.tag"
trained_model = "trained_model_crf.p"
previous_n = 0
next_n = 4
c = 10
prune = 5

helper.load_constants()
all_data, all_identifier = train.load_data(training_file)
train_split = .6
split_index = int(len(all_data)*train_split)
## TODO: Ask Karthik If i should randomize this
train_data, train_identifier = all_data[:split_index], all_identifier[:split_index]
test_data, test_identifier = all_data[split_index:], all_identifier[split_index:]
num_words, word_vocab = train.get_word_vocab(train_data, prune)    

trainXMatrix, trainYInts = train.get_feature_matrix_n(previous_n,next_n,train_data, num_words, word_vocab, helper.other_features)
trainX = trainXMatrix.toarray()
trainY = [train.int2tags[ int(y) ] for y in trainYInts]
trainModel()

In [ ]:
predictedY, testY, confidences, confidences_beam, info = predict()

In [144]:
print "TRAIN gold"
for tag in train.int2tags:
    numTag = sum([p==tag for p in trainY])
    print "Tag is ", tag, " WITH NUM :", numTag
    
print "TEST PREDICTED"
for tag in train.int2tags:
    numTag = sum([p==tag for p in predictedY])
    print "Tag is ", tag, " WITH NUM :", numTag

print "TEST gold"
for tag in train.int2tags:
    numTag = sum([p==tag for p in testY])
    print "Tag is ", tag, " WITH NUM :", numTag

TRAIN gold
Tag is  TAG  WITH NUM : 87829
Tag is  shooterName  WITH NUM : 425
Tag is  killedNum  WITH NUM : 173
Tag is  woundedNum  WITH NUM : 313
Tag is  city  WITH NUM : 1002
TEST PREDICTED
Tag is  TAG  WITH NUM : 89742
Tag is  shooterName  WITH NUM : 0
Tag is  killedNum  WITH NUM : 0
Tag is  woundedNum  WITH NUM : 0
Tag is  city  WITH NUM : 0
TEST gold
Tag is  TAG  WITH NUM : 88507
Tag is  shooterName  WITH NUM : 291
Tag is  killedNum  WITH NUM : 126
Tag is  woundedNum  WITH NUM : 215
Tag is  city  WITH NUM : 603


In [149]:
print info.transitions


{(u'TAG', u'woundedNum'): 1.545113, (u'TAG', u'TAG'): 4.423181, (u'city', u'killedNum'): -0.286106, (u'killedNum', u'TAG'): 1.614451, (u'woundedNum', u'killedNum'): -0.625218, (u'city', u'TAG'): 1.793026, (u'woundedNum', u'city'): -0.757178, (u'TAG', u'shooterName'): 1.481619, (u'city', u'woundedNum'): -0.112658, (u'shooterName', u'shooterName'): 2.983068, (u'shooterName', u'TAG'): 1.454725, (u'TAG', u'killedNum'): 0.926706, (u'TAG', u'city'): 2.18186, (u'city', u'city'): 2.71771, (u'woundedNum', u'TAG'): 1.624096}


In [141]:
def trainModel():
    ## extract features
    trainer = crf.Trainer(verbose=False)
    trainer.append(trainX, trainY)
#     trainer.set_params({
#         'c1': 0.0,   # coefficient for L1 penalty
#         'c2': 1e-3,  # coefficient for L2 penalty

#         # include transitions that are possible, but not observed
#         'feature.possible_transitions': True
#     })
    trainer.train(trained_model)

In [156]:
def predict():
    print "PREDICT"
    testXMatrix, testYInts =  train.get_feature_matrix_n(previous_n,next_n,test_data, num_words, word_vocab, helper.other_features)
    testX = testXMatrix.toarray()
    testY = [train.int2tags[ int(y) ] for y in testYInts]
    
    tagger = crf.Tagger()
    tagger.open(trained_model)
    
    predictedY = tagger.tag(testX)
    confidences = [tagger.marginal(predictedY[i],i) for i in range(len(predictedY))]
    confidences_beam = [ [tagger.marginal(tag, i)  for tag in train.int2tags]   for i in range(len(predictedY))]
    
    accuracy = sum([testY[i]==predictedY[i] for i in range(len(testY))]) * 1.0 / len(testY)
    print "OVERALL ACCURACY", accuracy
    
    for tag in train.int2tags:
        predicted_correct = sum( [predictedY[i] == tag and testY[i] == tag for i in range(len(testY))] )
        total_of_tag      = sum( [testY[i] == tag for i in range(len(testY))] )
        
        print "ACCURACY FOR ", tag, "IS ", predicted_correct * 1. / total_of_tag
    return predictedY, testY, confidences, confidences_beam, tagger.info()